## Kullm : 한국어를 위한 LLM

- github : https://github.com/nlpai-lab/KULLM
- hf model (12b model) : https://huggingface.co/nlpai-lab/kullm-polyglot-12.8b-v2
- 6b model : https://huggingface.co/nlpai-lab/kullm-polyglot-5.8b-v2

In [ ]:
!pip install -q transformers accelerate sentencepiece bitsandbytes

In [ ]:
import sagemaker
import transformers
print(sagemaker.__version__)
print(transformers.__version__)

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import os

local_model_path = Path("./pretrained-models")
local_model_path.mkdir(exist_ok=True)
model_name = "nlpai-lab/kullm-polyglot-12.8b-v2"
# model_name = "nlpai-lab/kullm-polyglot-5.8b-v2"
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model", "*.py"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

In [ ]:
print(f"Local model download path: {model_download_path}")

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
# init
tokenizer = AutoTokenizer.from_pretrained(model_download_path)
model = AutoModelForCausalLM.from_pretrained(
    model_download_path,
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    load_in_8bit=True
)
model.eval()

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
def infer(instruction="", input_text=""):
    prompt_format = "아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n{instruction}\n\n### 입력:\n{input}\n\n### 응답:\n"
    prompt = prompt_format.format(instruction=instruction, input=input_text)
    
    output = pipe(
        prompt,
        max_length=512,
        temperature=0.7,
        top_p=0.7,
        eos_token_id=2
    )
    s = output[0]["generated_text"]
    result = s.split("### 응답:")[1].strip()
    return result

In [ ]:
%%time
result = infer(input_text="새로 태어나는 아이를 위한 이름을 5개 추천해줘")
print(result)

In [ ]:
%%time
result = infer(input_text="삼성서울병원에 어떻게 가야되?")
print(result)

In [ ]:
%%time
result = infer(input_text="프롬프트 엔지니어링 잘 하는 법좀 알려주세요")
print(result)

In [ ]:
# To check eos token => <|endoftext|> : 2
# tokenizer("<|endoftext|>")

In [ ]:
# Release the model (clear cuda memory)
from numba import cuda
device = cuda.get_current_device()
device.reset()

### vLLM 을 활용한 테스트

vLLM을 사용하여 inference speed를 얼마나 높일 수 있는 지 테스트 진행
- vllm 0.1.1 기준으로 kullm 모델을 지원하지 않습니다. (kullm의 attention head size 가 256인데 지원되지 않음.)
- 이것은 코드를 수정해서 해결할 수 있습니다. 여기 이슈 (https://github.com/vllm-project/vllm/issues/302) 처럼 코드 수정 후 다시 빌드해서 하면 정상적으로 동작합니다.

In [ ]:
# !pip install -q vllm

In [ ]:
from vllm.vllm import LLM, SamplingParams

llm = LLM(model=model_download_path)

In [ ]:
sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.8,
    # max_tokens=128,
    max_tokens=512,
    stop=["<|endoftext|>"]
)

In [ ]:
def vllm_infer(instruction="", input_text=""):
    prompt_format = "아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n{instruction}\n\n### 입력:\n{input}\n\n### 응답:\n"
    prompt = prompt_format.format(instruction=instruction, input=input_text)
    
    output = llm.generate([prompt], sampling_params)
    generated_text = output[0].outputs[0].text
    
    return generated_text

In [ ]:
%%time
result = vllm_infer(input_text="프롬프트 엔지니어링 잘 하는 법좀 알려주세요")
print(result)

In [ ]:
%%time
result = vllm_infer(input_text="광주광역시 근처에 여행할 만한 곳좀 추천해 주세요.")
print(result)

### 테스트 결과
- 비슷한 parameter와 output token size에 그냥 HF transformers로 로딩했을 때와 비교해서 2배 가까이 inference 속도의 개선이 있는 것을 확인할 수 있습니다.
